In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, PeftConfig
from datasets import DatasetDict
from peft import prepare_model_for_kbit_training

In [3]:
dataset = load_dataset("miscovery/Math_CoT_Arabic_English_Reasoning", split="train")
# The dataset has fields: en_question, ar_question, en_answer, ar_answer, category, etc. :contentReference[oaicite:0]{index=0}

def make_prompt_and_target(example):
    # We'll make two versions: one English, one Arabic
    en_in = f"Language: English\nQuestion: {example['en_question']}\nAnswer:"
    ar_in = f"Language: Arabic\nQuestion: {example['ar_question']}\nAnswer:"

    en_out = example['en_answer']
    ar_out = example['ar_answer']

    return {
        "input_text": [en_in, ar_in],
        "target_text": [en_out, ar_out],
    }

paired = dataset.map(make_prompt_and_target, batched=False)

# Flatten the lists (because each example became 2)
def flatten_examples(ex):
    return {
        "input_text": ex["input_text"],
        "target_text": ex["target_text"],
    }

# Actually convert to a “flat” dataset
flat_dataset = paired.flatten()

In [4]:
print(flat_dataset[0])  # Check the first example

{'en_question': 'A baker makes 24 cupcakes. She sells half of them at $3 each, and then sells the remaining cupcakes at $2 each. How much money does she make in total?', 'ar_question': 'يخبز خباز 24 قطعة كعك. يبيع نصفها بسعر 3 دولارات للقطعة الواحدة، ثم يبيع القطع المتبقية بسعر 2 دولار للقطعة الواحدة. كم يكسب من المال في المجموع؟', 'en_answer': '#step 1: Half of 24 cupcakes is 24 / 2 = 12 cupcakes. #step 2: She sells 12 cupcakes at $3 each, so she makes 12 * 3 = $36. #step 3: She sells the remaining 12 cupcakes at $2 each, so she makes 12 * 2 = $24. #step 4: In total, she makes $36 + $24 = $60. #answer: $60', 'ar_answer': '#الخطوة 1: نصف 24 قطعة كعك هو 24 / 2 = 12 قطعة كعك. #الخطوة 2: تبيع 12 قطعة كعك بسعر 3 دولارات للقطعة الواحدة، لذا تكسب 12 * 3 = 36 دولارًا. #الخطوة 3: تبيع 12 قطعة الكعك المتبقية بسعر 2 دولار للقطعة الواحدة، لذا تكسب 12 * 2 = 24 دولارًا. #الخطوة 4: إجمالاً، تكسب 36 دولارًا + 24 دولارًا = 60 دولارًا. #الإجابة: 60 دولارًا', 'category': 'Mathematics - Arithmetic', 'en_

In [5]:
# 3. Load tokenizer
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# 4. Format into LLaMA Instruct format
def format_example(question, answer):
    """
    Reformats question/answer into LLaMA Instruct chat format.
    """
    return (
        "<|begin_of_text|>"
        "<|start_header|>user<|end_header|>\n"
        f"{question}\n"
        "<|start_header|>assistant<|end_header|>\n"
        f"{answer}"
    )


# 5. Tokenization with prompt masking
def tokenize_fn(examples):
    formatted_texts = []
    for inp, tgt in zip(examples["input_text"], examples["target_text"]):
        formatted = format_example(inp, tgt)
        formatted_texts.append(formatted)

    tokenized = tokenizer(
        formatted_texts,
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

    # Create labels and mask prompt
    labels = []
    for inp, tgt in zip(examples["input_text"], examples["target_text"]):
        full_text = format_example(inp, tgt)

        # Tokenize to get full input ids
        full_ids = tokenizer(
            full_text, truncation=True, max_length=1024
        )["input_ids"]

        # Tokenize only prompt part (question only)
        prompt_text = (
            "<|begin_of_text|>"
            "<|start_header|>user<|end_header|>\n"
            f"{inp}\n"
            "<|start_header|>assistant<|end_header|>\n"
        )
        prompt_ids = tokenizer(prompt_text)["input_ids"]

        # Mask prompt tokens
        lbl = full_ids.copy()
        prompt_len = len(prompt_ids)
        lbl[:prompt_len] = [-100] * prompt_len

        # Pad labels to max_length
        if len(lbl) < 1024:
            lbl = lbl + [-100] * (1024 - len(lbl))
        else:
            lbl = lbl[:1024]

        labels.append(lbl)

    tokenized["labels"] = labels
    return tokenized


# 6. Apply tokenization & clean dataset columns
tokenized = flat_dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=[
        "input_text", "target_text",
        "en_question", "ar_question",
        "en_answer", "ar_answer",
        "category",
        "en_q_word", "ar_q_word",
        "en_a_word", "ar_a_word"
    ]
)


In [6]:
print(tokenized[0])  

{'input_ids': [128000, 128000, 27, 91, 2527, 8932, 91, 29, 882, 27, 91, 408, 8932, 91, 397, 681, 14126, 25, 6498, 1734, 14924, 25, 362, 76928, 3727, 220, 1187, 88993, 13, 3005, 31878, 4376, 315, 1124, 520, 400, 18, 1855, 11, 323, 1243, 31878, 279, 9861, 88993, 520, 400, 17, 1855, 13, 2650, 1790, 3300, 1587, 1364, 1304, 304, 2860, 33720, 77, 16533, 17898, 364, 14126, 25, 35217, 1734, 14924, 25, 74374, 115231, 40797, 75415, 125516, 220, 1187, 105077, 101454, 88041, 24102, 32173, 13, 74374, 116215, 51343, 106641, 100338, 111151, 103679, 220, 18, 103744, 103363, 100728, 103380, 101454, 105155, 100528, 102539, 69885, 108126, 74374, 116215, 100864, 121380, 54579, 100936, 117741, 111151, 103679, 220, 17, 119303, 100728, 103380, 101454, 105155, 100528, 102539, 13, 109025, 110333, 101108, 64337, 54579, 32482, 78373, 54579, 103969, 101735, 59, 77, 16533, 25, 4532, 27, 91, 2527, 8932, 91, 29, 78191, 27, 91, 408, 8932, 91, 397, 681, 2, 9710, 220, 16, 25, 26924, 315, 220, 1187, 88993, 374, 220, 118

In [7]:
split1 = tokenized.train_test_split(test_size=0.2, seed=42)

# temp (20%) → validation (10%) + test (10%)
split2 = split1['test'].train_test_split(test_size=0.5, seed=42)

# Combine into DatasetDict
final_dataset = DatasetDict({
    "train": split1['train'],       
    "validation": split2['train'],  
    "test": split2['test']           
})

In [8]:
print(len(final_dataset['train']))       # Number of training examples
print(len(final_dataset['validation']))  # Number of validation examples
print(len(final_dataset['test']))        # Number of test examples

2267
283
284


In [9]:
# 5. Load model with bitsandbytes for quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # Best for QLoRA
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# Disable cache BEFORE attaching LoRA
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)


# 6. Set up LoRA via PEFT
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
    ],
)

model = get_peft_model(model, lora_config)

# Enable gradient checkpointing ONLY AFTER LoRA is attached
model.gradient_checkpointing_enable()

# Print trainable parameters to confirm it's working
model.print_trainable_parameters()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 9,175,040 || all params: 3,221,924,864 || trainable%: 0.2848


In [10]:
training_args = TrainingArguments(
    output_dir="./llama_mathcot_qlora",
    # Training
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    fp16=False,
    bf16=True,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",

    # Evaluation
    evaluation_strategy="steps",   
    eval_steps=200,                
    eval_on_start=True,
    logging_steps=50,
    logging_first_step=True,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # WandB Logging 
    report_to="wandb",             
    run_name="llama_mathcot_qlora",

    # Saving 
    save_total_limit=3,

    # Other 
    max_grad_norm=1.0,
    warmup_ratio=0.03,
    dataloader_num_workers=4,
)


c:\Anacoda\envs\py330\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
import wandb
wandb.login()

wandb: Currently logged in as: islam-hitham (islam-hitham-king-salman-international-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["validation"],
)

In [12]:
trainer.train()

  0%|          | 0/849 [00:00<?, ?it/s]

c:\Anacoda\envs\py330\lib\site-packages\transformers\models\llama\modeling_llama.py:602: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/283 [00:00<?, ?it/s]

{'eval_loss': 0.8697879910469055, 'eval_runtime': 143.7257, 'eval_samples_per_second': 1.969, 'eval_steps_per_second': 1.969, 'epoch': 0}
{'loss': 0.9605, 'grad_norm': 0.9188120365142822, 'learning_rate': 7.692307692307694e-06, 'epoch': 0.0}
{'loss': 0.5789, 'grad_norm': 0.36892950534820557, 'learning_rate': 0.0001995806382799115, 'epoch': 0.18}
{'loss': 0.4176, 'grad_norm': 0.38518038392066956, 'learning_rate': 0.00019603681998124953, 'epoch': 0.35}
{'loss': 0.3953, 'grad_norm': 0.49780428409576416, 'learning_rate': 0.00018900514150473838, 'epoch': 0.53}
{'loss': 0.3803, 'grad_norm': 0.30692413449287415, 'learning_rate': 0.00017874097897951737, 'epoch': 0.71}


  0%|          | 0/283 [00:00<?, ?it/s]

...

  0%|          | 0/283 [00:00<?, ?it/s]

{'eval_loss': 0.3202255368232727, 'eval_runtime': 140.5353, 'eval_samples_per_second': 2.014, 'eval_steps_per_second': 2.014, 'epoch': 2.12}
{'loss': 0.2252, 'grad_norm': 0.4545200765132904, 'learning_rate': 2.749103496282306e-05, 'epoch': 2.29}
{'loss': 0.2375, 'grad_norm': 0.41093361377716064, 'learning_rate': 1.5743564964878867e-05, 'epoch': 2.47}
{'loss': 0.2335, 'grad_norm': 0.4645814001560211, 'learning_rate': 7.056115743798308e-06, 'epoch': 2.65}
{'loss': 0.2505, 'grad_norm': 0.46202537417411804, 'learning_rate': 1.7441976235509917e-06, 'epoch': 2.82}


  0%|          | 0/283 [00:00<?, ?it/s]

{'eval_loss': 0.31805771589279175, 'eval_runtime': 141.0233, 'eval_samples_per_second': 2.007, 'eval_steps_per_second': 2.007, 'epoch': 2.82}
{'train_runtime': 10770.7902, 'train_samples_per_second': 0.631, 'train_steps_per_second': 0.079, 'train_loss': 0.32036555922635174, 'epoch': 3.0}


TrainOutput(global_step=849, training_loss=0.32036555922635174, metrics={'train_runtime': 10770.7902, 'train_samples_per_second': 0.631, 'train_steps_per_second': 0.079, 'total_flos': 1.1800933882763674e+17, 'train_loss': 0.32036555922635174, 'epoch': 2.996029995588884})

In [13]:
# Save adapter / LoRA weights
model.save_pretrained("./llama_mathcot_qlora_adapter")

In [13]:
# Inference (load adapter + model again)
peft_config = PeftConfig.from_pretrained("./llama_mathcot_qlora_adapter")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
model = get_peft_model(model, peft_config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_answer(question: str, lang: str):
    prompt = f"Language: {lang}\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(out[0], skip_special_tokens=True)

# Example:
print(generate_answer("If 4 apples cost $2, how much do 10 apples cost?", "English"))
print(generate_answer("إذا كانت تكلفة 4 تفاحات 2 دولار، فما هي تكلفة 10 تفاحات؟", "Arabic"))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Anacoda\envs\py330\lib\site-packages\transformers\models\llama\modeling_llama.py:602: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Language: English
Question: If 4 apples cost $2, how much do 10 apples cost?
Answer: $1
Explanation: 4 apples cost $2, so 1 apple costs $2/4 = $0.50. 10 apples cost $0.50 x 10 = $5. The answer is $5, not $1. I apologize for the mistake earlier.
The best answer is $5.
Language: Arabic
Question: إذا كانت تكلفة 4 تفاحات 2 دولار، فما هي تكلفة 10 تفاحات؟
Answer: إذا كانت تكلفة 4 تفاحات 2 دولار، فما هي تكلفة 10 تفاحات؟
الجواب: 20
الشرح: هذه المرة، سوف نستخدم المعدل الذي يعد التكلفة من 4 إلى 10 تفاحات، مع التأكيد على أن 10 تفاحات هي 10/4 = 2.5 أوقية، وتكلفة 2.5 أوقية هي 2.5 * 2 = 5 دولار. 
الشرح: إذا كانت تكلفة 4 تفاحات 2 دولار، فما هي تكلفة 10 تفاحات؟

الجواب: 20
الشرح: هذه المرة، سوف نستخدم المعدل الذي يعد التكلفة من 4 إلى 10 تفاحات، مع التأكيد على أن 10 تفاحات هي 10/4 = 2
